<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasci 266 final project - enhancement notebook - BART

**Description:**

**Structure of the notebook:**

**Expected results:**





## Setup

In [ ]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q pandas
!pip install -q rouge_score
!pip install -q nltk
!pip install bert_score
!pip install sentence-transformers


In [ ]:
#import evaluate
import torch
my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
my_device

#from transformers import T5Tokenizer, TFT5ForConditionalGeneration
#from transformers import BertTokenizer, BertModel

import evaluate
import numpy as np
from transformers import pipeline

In [ ]:
'''
#!pip install git+https://github.com/sleighton2022/datasci266-final-project.git
!wget https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel
'''

--2024-07-24 04:38:38--  https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11185 (11K) [text/plain]
Saving to: ‘summary_utils.py.1’

summary_utils.py.1  100%[===================>]  10.92K  --.-KB/s    in 0s      

2024-07-24 04:38:38 (113 MB/s) - ‘summary_utils.py.1’ saved [11185/11185]



In [ ]:
'''
# Load dataset
dataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)
sampled_dataset = dataset_manager.load_sampled_dataset()
'''

'\n# Load dataset\ndataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)\nsampled_dataset = dataset_manager.load_sampled_dataset()\n'

In [ ]:
from datasets import load_dataset
xsum_dataset = load_dataset("xsum", trust_remote_code=True)

In [ ]:
train_data = xsum_dataset['train']
train_data = train_data.shuffle(seed=42)
test_data = xsum_dataset['test']
test_data = test_data.shuffle(seed=42)
print(len(train_data))
print(len(test_data))
print(train_data[0])


204045
11334
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to what they call a "com

In [ ]:
#subsample training dataset
sample_size = 500
train_dataset = train_data.select(range(sample_size))
train_dataset

#set test dataset for bart and refine bart models
test_dataset = test_data.select(range(200))

In [ ]:
#global params
TOK_MAX_LEN = 1024
SUMMARY_MAX_LEN = 80
SUMMARY_MIN_LEN = 30


# 3. Model Enhancement Experiments
- two steps: 1. page rank 2. T5/Pegasus
- two steps: 1. cluster by topic 2. T5/pegasus
- Fine tuning layers：  first layer,  last layer (task specific head)
- LoRA

BERTSUM: An extension of BERT for summarization that uses a transformer encoder to produce contextual embeddings and a classifier to select sentences.
PEGASUS: A transformer-based model specifically designed for summarization tasks. It is trained with a self-supervised objective that masks entire sentences.
T5 (Text-to-Text Transfer Transformer): Treats all NLP tasks as text-to-text problems, including summarization. It has been shown to perform well on various summarization benchmarks.
BART (Bidirectional and Auto-Regressive Transformers): A transformer model trained to reconstruct corrupted text, making it suitable for both extractive and abstractive summarization.
GPT-3: Though primarily a generative language model, it can also perform summarization tasks by generating summaries based on the input text.
Example of Using a Transformer Model for Summarization


# BART Models

first run on pre-trained only model without fine tuning

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the BART tokenizer and model

btokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bmodel = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
'''
btokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
bmodel = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

btokenizer = BartTokenizer.from_pretrained('facebook/bart-base-xsum')
bmodel = BartForConditionalGeneration.from_pretrained('facebook/bart-base-xsum')
'''
bmodel = bmodel.to(my_device)

bart_summaries = []
for article in test_dataset['document']:
    inputs = btokenizer(article,  return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(my_device)
    summary_ids = bmodel.generate(inputs["input_ids"], min_length = 25, max_length= 60,  num_beams=4, do_sample = True)
    candidate = btokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    bart_summaries.append(candidate)

# fine tuning

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = [article for article in examples["document"]]
    model_inputs = btokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    with btokenizer.as_target_tokenizer():
        labels = btokenizer(examples["summary"], max_length= 80, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
bart_tok_train = train_dataset.map(tokenize_function, batched=True)

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# Freeze all layers except the first two encoder and decoder layers
for param in bmodel.parameters():
    param.requires_grad = False

for param in bmodel.model.encoder.layers[:2].parameters():
    param.requires_grad = True

for param in bmodel.model.decoder.layers[:2].parameters():
    param.requires_grad = True

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs= 10,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=bmodel,
    args=training_args,
    train_dataset= bart_tok_train,
    eval_dataset= bart_tok_train
)

# Fine-tune the model
trainer.train()

metrics = trainer.evaluate()
print(metrics)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,6.052731
2,No log,5.166945
3,No log,4.878640


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'eval_loss': 4.8786396980285645, 'eval_runtime': 15.2672, 'eval_samples_per_second': 6.55, 'eval_steps_per_second': 3.275, 'epoch': 3.0}


In [ ]:
eft_bart_summaries = []
for article in test_dataset['document']:
    inputs = btokenizer(article,  return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(my_device)
    summary_ids = bmodel.generate(inputs["input_ids"], max_length=SUMMARY_MAX_LEN,  num_beams=4, do_sample = True)
    candidate = btokenizer.decode(summary_ids[0],skip_special_tokens=True)
    eft_bart_summaries.append(candidate)

In [ ]:
model_name = 'facebook/bart-large-xsum'
bart_xsum_model = BartForConditionalGeneration.from_pretrained(model_name)
bart_xsum_tokenizer = BartTokenizer.from_pretrained(model_name)
config = bart_xsum_model.config
max_position_embeddings_bartsum = config.max_position_embeddings

In [ ]:
summarizer = pipeline("summarization", model= bart_xsum_model, tokenizer = bart_xsum_tokenizer, device=-1)
batch_input_ids = bart_xsum_tokenizer(test_dataset['document'], return_tensors="pt", truncation=True, padding="max_length") #, max_length=max_position_embeddings_bartsum,
decoded_texts = bart_xsum_tokenizer.batch_decode(batch_input_ids['input_ids'], skip_special_tokens=True)
batch_summary = summarizer(
    decoded_texts,
    max_length= SUMMARY_MAX_LEN,
    min_length= SUMMARY_MIN_LEN,
    length_penalty= 2.0,
    num_beams= 4,
    early_stopping= True
)
xsum_bart_summaries = [x['summary_text'] for x in batch_summary]

Your max_length is set to 80, but your input_length is only 70. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your max_length is set to 80, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 80, but your input_length is only 52. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 80, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max

In [ ]:
#check generated summary vs. reference
for i in range(len(test_dataset[:3])):

  print(test_dataset['summary'][i])
  print(bart_summaries[i])
  print(eft_bart_summaries[i])
  print(xsum_bart_summaries[i])
  print()

A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Sarah Johnson was one of 21 women in a minibus hit by a lorry on the M62. Her friend Bethany Jones, 18, was killed in the crash in April 2013. Ms Johnson broke her shoulder, back and pelvis and spent time in hospital.
Sarah Johnson was one of 21 women heading to Liverpool when their minibus was hit by a lorry on the M62. Her friend Bethany Jones, 18, was killed while Ms Johnson and several others were badly hurt. Ms Johnson, who broke her shoulder, back and pelvis, said the help she received from a charity led her to want to support others.
A woman who was seriously injured in a motorway crash which killed her friend has said she wants to help other victims of major trauma to recover.

A Tudor manor house has reopened following a £2.2m makeover.
Bramall Hall dates back to the reign of William the Conqueror. The estate has been under the ownership of just thr

In [ ]:
'''
#evaluation
summary_evaluator = SummaryEvaluator()
references = test_dataset['summary']
avg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, bart_summaries)
avg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, eft_bart_summaries)
#avg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, xsum_bart_summaries)
'''

"\n#evaluation\nsummary_evaluator = SummaryEvaluator()\nreferences = test_dataset['summary']\navg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, bart_summaries)\navg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, eft_bart_summaries)\n#avg_rouge_scores, avg_bleu_score, avg_bert_score, avg_vector_similarity = summary_evaluator.evaluate(references, xsum_bart_summaries)\n"

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')

references = test_dataset['summary']
print(bleu.compute(predictions=bart_summaries, references=references))
print(rouge.compute(predictions=bart_summaries, references=references))
print()
print(bleu.compute(predictions=eft_bart_summaries, references=references))
print(rouge.compute(predictions=eft_bart_summaries, references=references))
print()
print(bleu.compute(predictions=xsum_bart_summaries, references=references))
print(rouge.compute(predictions=xsum_bart_summaries, references=references))


{'bleu': 0.01581709133323252, 'precisions': [0.16147449422862106, 0.02456408298332697, 0.0069217709285621, 0.0022797371597157033], 'brevity_penalty': 1.0, 'length_ratio': 1.7634055592033269, 'translation_length': 8057, 'reference_length': 4569}
{'rouge1': 0.2049530846495754, 'rouge2': 0.0363046061986633, 'rougeL': 0.1445025336344798, 'rougeLsum': 0.14425907424312273}

{'bleu': 0.008778589167393392, 'precisions': [0.13738719004643826, 0.017390528850441452, 0.003359665849450649, 0.0007398501803384815], 'brevity_penalty': 1.0, 'length_ratio': 2.49792077040928, 'translation_length': 11413, 'reference_length': 4569}
{'rouge1': 0.19401733295353862, 'rouge2': 0.02858151556938742, 'rougeL': 0.12854805899940663, 'rougeLsum': 0.12873860534021983}

{'bleu': 0.12862279939221086, 'precisions': [0.36580690135935867, 0.1570964247020585, 0.08898463844136381, 0.053522771506422734], 'brevity_penalty': 1.0, 'length_ratio': 1.2558546727949222, 'translation_length': 5738, 'reference_length': 4569}
{'rouge1